In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import joblib

from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
import spacy
import gensim
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize

import statistics


import re
from os import path
from PIL import Image
import plotly
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.collocations import *
nltk.download('averaged_perceptron_tagger')

from textblob import TextBlob

import seaborn as sns

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sameer/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
def normalize_list(data):
    x_min = min(data)
    x_max = max(data)
    if x_max-x_min == 0:
        return None
    return [(x-x_min)/(x_max-x_min) for x in data]

In [3]:
df = pd.read_csv('./tesla_sentiments.csv', encoding='utf-8')

In [4]:
df.head()

,Unnamed: 0,Company,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Job_Title,Job_Location,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Advice_to_Management,Pros_And_Cons,Review_Title_sentiment_score_vader,Pros_And_Cons_sentiment_score_vader
0,0,Tesla,3.0,1,more than 5 years,"A unique company, but it’s hard to last more t...",27-04-17,Anonymous Employee,"Fremont, CA",0,-1,1,Completely casual dress code\r\n\r\nFlexibilit...,"Rare to be recognized, let alone thanked, for ...",I was debating whether I approve of the CEO or...,Completely casual dress code\r\n\r\nFlexibilit...,-0.15,1.00
1,1,Tesla,4.0,1,less than 1 year,Good place to start in tech,29-03-22,Data Annotation Specialist,"Buffalo, NY",2,2,1,"Good hours, benefits and work environment",No overtime. They slightly mislead when I orig...,Give at least the top performers the chance to...,"Good hours, benefits and work environmentNo ov...",0.44,0.18
2,2,Tesla,5.0,1,more than 1 year,Excellent company to work for all around,03-03-22,Software Engineer,"Stamford, CT",2,2,2,"- Great culture, everyone is extremely positiv...",- Long hours\r\n- Quite challenging (but rewar...,NaN,"- Great culture, everyone is extremely positiv...",0.57,0.50
3,3,Tesla,5.0,1,NaN,Excellent company to work for,03-03-22,Software Engineer,"Stamford, CT",2,2,2,"- Great culture, everyone is extremely positiv...",- Long hours\r\n- Quite challenging (but rewar...,NaN,"- Great culture, everyone is extremely positiv...",0.57,0.50
4,4,Tesla,3.0,0,less than 1 year,Hero to Zero due to a very bad boss,29-03-22,Service Manager,"Fremont, CA",2,2,2,Tesla is such an awesome company. Was very exc...,Manager was a narcissist psychopath who after ...,"actually, enforce HR rules and take action for...",Tesla is such an awesome company. Was very exc...,-0.05,0.18


In [5]:
df.columns

Index(['Unnamed: 0', 'Company', 'Star_Rating', 'Current_Employee', 'Work_Duration', 'Review_Title', 'Date_Posted', 'Job_Title', 'Job_Location', 'Recommend', 'CEO_Approval', 'Business_Outlook', 'Pros', 'Cons', 'Advice_to_Management', 'Pros_And_Cons', 'Review_Title_sentiment_score_vader', 'Pros_And_Cons_sentiment_score_vader'], dtype='object')

In [6]:
df.drop(['Unnamed: 0', 'Company'], axis=1, inplace=True)

In [7]:
df.head()

,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Job_Title,Job_Location,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Advice_to_Management,Pros_And_Cons,Review_Title_sentiment_score_vader,Pros_And_Cons_sentiment_score_vader
0,3.0,1,more than 5 years,"A unique company, but it’s hard to last more t...",27-04-17,Anonymous Employee,"Fremont, CA",0,-1,1,Completely casual dress code\r\n\r\nFlexibilit...,"Rare to be recognized, let alone thanked, for ...",I was debating whether I approve of the CEO or...,Completely casual dress code\r\n\r\nFlexibilit...,-0.15,1.00
1,4.0,1,less than 1 year,Good place to start in tech,29-03-22,Data Annotation Specialist,"Buffalo, NY",2,2,1,"Good hours, benefits and work environment",No overtime. They slightly mislead when I orig...,Give at least the top performers the chance to...,"Good hours, benefits and work environmentNo ov...",0.44,0.18
2,5.0,1,more than 1 year,Excellent company to work for all around,03-03-22,Software Engineer,"Stamford, CT",2,2,2,"- Great culture, everyone is extremely positiv...",- Long hours\r\n- Quite challenging (but rewar...,NaN,"- Great culture, everyone is extremely positiv...",0.57,0.50
3,5.0,1,NaN,Excellent company to work for,03-03-22,Software Engineer,"Stamford, CT",2,2,2,"- Great culture, everyone is extremely positiv...",- Long hours\r\n- Quite challenging (but rewar...,NaN,"- Great culture, everyone is extremely positiv...",0.57,0.50
4,3.0,0,less than 1 year,Hero to Zero due to a very bad boss,29-03-22,Service Manager,"Fremont, CA",2,2,2,Tesla is such an awesome company. Was very exc...,Manager was a narcissist psychopath who after ...,"actually, enforce HR rules and take action for...",Tesla is such an awesome company. Was very exc...,-0.05,0.18


In [8]:
df['Date_Posted'] = pd.to_datetime(df['Date_Posted'], dayfirst=True)

In [9]:
quarters = pd.Series(pd.date_range('2012-01-01', '2022-01-01', freq='QS'))

In [10]:
len(quarters)

41

In [11]:
for i, row in df.iterrows():
	if isinstance(row['Work_Duration'], str):
		if 'less' in row['Work_Duration']:
			years = [int(i) for i in row['Work_Duration'].split() if i.isdigit()]
			df.at[i, 'Work_Duration'] = years[0] - 1
			# row['Work_Duration'] = years[0] - 1
		elif 'more' in row['Work_Duration']:
			years = [int(i) for i in row['Work_Duration'].split() if i.isdigit()]
			df.at[i, 'Work_Duration'] = years[0]
			# row['Work_Duration'] = years[0]

In [12]:
df.head()

,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Job_Title,Job_Location,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Advice_to_Management,Pros_And_Cons,Review_Title_sentiment_score_vader,Pros_And_Cons_sentiment_score_vader
0,3.0,1,5,"A unique company, but it’s hard to last more t...",2017-04-27,Anonymous Employee,"Fremont, CA",0,-1,1,Completely casual dress code\r\n\r\nFlexibilit...,"Rare to be recognized, let alone thanked, for ...",I was debating whether I approve of the CEO or...,Completely casual dress code\r\n\r\nFlexibilit...,-0.15,1.00
1,4.0,1,0,Good place to start in tech,2022-03-29,Data Annotation Specialist,"Buffalo, NY",2,2,1,"Good hours, benefits and work environment",No overtime. They slightly mislead when I orig...,Give at least the top performers the chance to...,"Good hours, benefits and work environmentNo ov...",0.44,0.18
2,5.0,1,1,Excellent company to work for all around,2022-03-03,Software Engineer,"Stamford, CT",2,2,2,"- Great culture, everyone is extremely positiv...",- Long hours\r\n- Quite challenging (but rewar...,NaN,"- Great culture, everyone is extremely positiv...",0.57,0.50
3,5.0,1,NaN,Excellent company to work for,2022-03-03,Software Engineer,"Stamford, CT",2,2,2,"- Great culture, everyone is extremely positiv...",- Long hours\r\n- Quite challenging (but rewar...,NaN,"- Great culture, everyone is extremely positiv...",0.57,0.50
4,3.0,0,0,Hero to Zero due to a very bad boss,2022-03-29,Service Manager,"Fremont, CA",2,2,2,Tesla is such an awesome company. Was very exc...,Manager was a narcissist psychopath who after ...,"actually, enforce HR rules and take action for...",Tesla is such an awesome company. Was very exc...,-0.05,0.18


In [13]:
df.drop('Job_Title', axis=1, inplace=True)

In [14]:
df.drop('Advice_to_Management', axis=1, inplace=True)

In [15]:
df.drop('Job_Location', axis=1, inplace=True)

In [16]:
df['Work_Duration'].fillna(df['Work_Duration'].mean(),inplace=True)

# other value aggregates

In [17]:
star_rating_aggregates = dict()
title_senti_aggregates = dict()
pnc_senti_aggregates = dict()
work_duration_aggregates = dict()

current_employee_aggregates = dict()
recommend_aggregates = dict()
ceo_approval_aggregates = dict()
business_outlook_aggregates = dict()



# percentages
recommend_percentages = dict()
ceo_percentages = dict()
business_percentages = dict()
counts = dict()
count = 0

In [18]:

for q in quarters:
	star_rating_aggregates[f'Q{count+1}'] = 0
	title_senti_aggregates[f'Q{count+1}'] = 0
	pnc_senti_aggregates[f'Q{count+1}'] = 0
	work_duration_aggregates[f'Q{count+1}'] = 0


	current_employee_aggregates[f'Q{count+1}'] = []
	recommend_aggregates[f'Q{count+1}'] = []
	ceo_approval_aggregates[f'Q{count+1}'] = []
	business_outlook_aggregates[f'Q{count+1}'] = []

	recommend_percentages[f'Q{count+1}'] = dict()
	ceo_percentages[f'Q{count+1}'] = dict()
	business_percentages[f'Q{count+1}'] = dict()
	for i in range(-1, 3):
		recommend_percentages[f'Q{count+1}'][i] = 0 
		ceo_percentages[f'Q{count+1}'][i] = 0
		business_percentages[f'Q{count+1}'][i] = 0


	counts[f'Q{count+1}'] = 0
	count += 1

for idx, row in df.iterrows():
	for i in range(len(quarters)):
		if i < len(quarters) - 1:
			if row['Date_Posted'] >= quarters[i] and row['Date_Posted'] < quarters[i+1]:
				star_rating_aggregates[f'Q{i+1}'] += row['Star_Rating']
				title_senti_aggregates[f'Q{i+1}'] += row['Review_Title_sentiment_score_vader']
				pnc_senti_aggregates[f'Q{i+1}'] += row['Pros_And_Cons_sentiment_score_vader']
				work_duration_aggregates[f'Q{i+1}'] += row['Work_Duration']


				current_employee_aggregates[f'Q{i+1}'].append(row['Current_Employee'])
				recommend_aggregates[f'Q{i+1}'].append(row['Recommend'])
				ceo_approval_aggregates[f'Q{i+1}'].append(row['CEO_Approval'])
				business_outlook_aggregates[f'Q{i+1}'].append(row['Business_Outlook'])

				recommend_percentages[f'Q{i+1}'][row['Recommend']] += 1
				ceo_percentages[f'Q{i+1}'][row['CEO_Approval']] += 1
				business_percentages[f'Q{i+1}'][row['Business_Outlook']] += 1
				
				counts[f'Q{i+1}'] += 1

In [19]:

avg_star_ratings = []
avg_title_senti = []
avg_pnc_senti = []
avg_work_duration = []

avg_current_employee = []
avg_recommend = []
avg_ceo_approval = []
avg_business_outlook = []

# percentages 

recommend_minus_percentages = []
ceo_minus_percentages = []
business_minus_percentages = []

recommend_unknown_percentages = []
ceo_unknown_percentages = []
business_unknown_percentages = []

recommend_same_percentages = []
ceo_same_percentages = []
business_same_percentages = []

recommend_plus_percentages = []
ceo_plus_percentages = []
business_plus_percentages = []


quarts = []


In [20]:
for quarter, value in star_rating_aggregates.items():

	if counts[quarter] != 0:

		avg_star_ratings.append(star_rating_aggregates[quarter] / counts[quarter])
		avg_title_senti.append(title_senti_aggregates[quarter] / counts[quarter])
		avg_pnc_senti.append(pnc_senti_aggregates[quarter] / counts[quarter])
		avg_work_duration.append(work_duration_aggregates[quarter] / counts[quarter])


		avg_current_employee.append(statistics.mode(current_employee_aggregates[quarter]))
		avg_recommend.append(statistics.mode(recommend_aggregates[quarter]))
		avg_ceo_approval.append(statistics.mode(ceo_approval_aggregates[quarter]))
		avg_business_outlook.append(statistics.mode(business_outlook_aggregates[quarter]))

		recommend_minus_percentages.append(recommend_percentages[quarter][-1] / counts[quarter])
		ceo_minus_percentages.append(ceo_percentages[quarter][-1] / counts[quarter])
		business_minus_percentages.append(business_percentages[quarter][-1] / counts[quarter])

		recommend_unknown_percentages.append(recommend_percentages[quarter][0] / counts[quarter])
		ceo_unknown_percentages.append(ceo_percentages[quarter][0] / counts[quarter])
		business_unknown_percentages.append(business_percentages[quarter][0] / counts[quarter])

		recommend_same_percentages.append(recommend_percentages[quarter][1] / counts[quarter])
		ceo_same_percentages.append(ceo_percentages[quarter][1] / counts[quarter])
		business_same_percentages.append(business_percentages[quarter][1] / counts[quarter])

		recommend_plus_percentages.append(recommend_percentages[quarter][2] / counts[quarter])
		ceo_plus_percentages.append(ceo_percentages[quarter][2] / counts[quarter])
		business_plus_percentages.append(business_percentages[quarter][2] / counts[quarter])
		
		quarts.append(quarter)

<hr>

In [21]:
def clean_text(text):
	text=re.sub(r"\?", ".", text)
	text=re.sub(r"\!", ".", text)
	text=re.sub(r'([.])\1+', r'\1', text)
	rexp=r"\.(?=[A-Za-z]{1,15})"
	text=re.sub(rexp, ". ", text)
	return text

#this function extracts lemmas of nouns, verbs and adjectives
def lemma_words_pos_filtered(text):
    word_list=[]
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(text)
    pos_tokens=nltk.pos_tag(tokens)
    for pos in pos_tokens:
        if (pos[1].startswith('N')):
            word_list=word_list+[lemmatizer.lemmatize(pos[0].lower(), wordnet.NOUN)]
        if (pos[1].startswith('V')):
            word_list=word_list+[lemmatizer.lemmatize(pos[0].lower(), wordnet.VERB)]
        if (pos[1].startswith('J')):
            word_list=word_list+[lemmatizer.lemmatize(pos[0].lower(), wordnet.ADJ)]
    word_list=[word for word in word_list if word not in  
                                    stopwords.words('english') ]
    return  " ".join(word_list)

#to keep only unique tuples
def unique_tuples(list_of_tuples):
     list_ = [tuple(sorted(t)) for t in list_of_tuples]
     list_ = list(set(list_))
     return list_

def findbigramsintext(text):
	mylist=nltk.word_tokenize(text)
	list1=[x for x in mylist]
	feature_list = []
	for i in range(length_features2):
	# In each iteration, add an empty list to the main list      
			feature_list.append([])
	i=0
	# bigram_features is the list of unique set of bigrams found after collocation
	for l in bigram_features:
			list2=[x for x in l]
			result =  all(elem in list1  for elem in list2)
			if result: 
				feature_list[i].append(' '.join(list2))
			i=i+1
	return feature_list


def keepnonempty(list1):
       mylist= [x for x in list1 if x != []]
       return mylist
def flatten_list(row_list):
        l = [item for inner_list in row_list for item in inner_list]
        return l


In [22]:
import nltk
nltk.download('punkt')
nltk.download('wordnet') 
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/sameer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/sameer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sameer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# text cleaning

In [23]:

df['Cleaned_Pros_and_Cons_Text'] = df['Pros_And_Cons'].apply(sent_tokenize)

In [24]:
len(df)

5016

In [25]:
df = df.explode('Cleaned_Pros_and_Cons_Text', ignore_index=False)

In [26]:
len(df)

12470

In [27]:
df['lemma_words_pos_filtered'] = df['Cleaned_Pros_and_Cons_Text'].apply(lemma_words_pos_filtered)

In [28]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words( " ".join(df['lemma_words_pos_filtered']).split(),window_size=5)
#keeping bigrams that occur at least 5 times
finder.apply_freq_filter(5)
bigram_list_tuples=finder.nbest(bigram_measures.likelihood_ratio, 20)

In [29]:
bigram_list_tuples

[('life', 'balance'),
 ('long', 'hour'),
 ('fast', 'pace'),
 ('stock', 'option'),
 ('“', '”'),
 ('bay', 'area'),
 ('work', 'life'),
 ('work/life', 'balance'),
 ('none', 'none'),
 ('work-life', 'balance'),
 ('cut', 'edge'),
 ('elon', 'musk'),
 ('upper', 'management'),
 ('free', 'cereal'),
 ('work', 'balance'),
 ('*', '*'),
 ('day', 'week'),
 ('great', 'benefit'),
 ('end', 'quarter'),
 ('change', 'world')]

In [30]:
bigram_features=unique_tuples(bigram_list_tuples)
length_features2=len(bigram_features)

In [31]:
df['bigram_list'] = df['lemma_words_pos_filtered'].apply(findbigramsintext)

In [32]:
df['bigrams']=df['bigram_list'].apply(keepnonempty)
df['bigrams']=df['bigrams'].apply(flatten_list)

In [33]:
#Sentiment scores are calculated using libraries TextBlob and Vader
df['polarity'] = df['Cleaned_Pros_and_Cons_Text'].apply(lambda x: TextBlob(x).sentiment[0])

In [34]:
df.head()

,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Pros_And_Cons,Review_Title_sentiment_score_vader,Pros_And_Cons_sentiment_score_vader,Cleaned_Pros_and_Cons_Text,lemma_words_pos_filtered,bigram_list,bigrams,polarity
0,3.0,1,5.0,"A unique company, but it’s hard to last more t...",2017-04-27,0,-1,1,Completely casual dress code\r\n\r\nFlexibilit...,"Rare to be recognized, let alone thanked, for ...",Completely casual dress code\r\n\r\nFlexibilit...,-0.15,1.0,Completely casual dress code\r\n\r\nFlexibilit...,casual dress code flexibility work home need i...,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.144048
0,3.0,1,5.0,"A unique company, but it’s hard to last more t...",2017-04-27,0,-1,1,Completely casual dress code\r\n\r\nFlexibilit...,"Rare to be recognized, let alone thanked, for ...",Completely casual dress code\r\n\r\nFlexibilit...,-0.15,1.0,Many “beautiful people” here (male and female).,many “ beautiful people ” male female,"[[], [], [], [], [], [], [], [], [], [], [], [...",[“ ”],0.337500
0,3.0,1,5.0,"A unique company, but it’s hard to last more t...",2017-04-27,0,-1,1,Completely casual dress code\r\n\r\nFlexibilit...,"Rare to be recognized, let alone thanked, for ...",Completely casual dress code\r\n\r\nFlexibilit...,-0.15,1.0,Lots of eye candy.,lot eye candy,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.000000
0,3.0,1,5.0,"A unique company, but it’s hard to last more t...",2017-04-27,0,-1,1,Completely casual dress code\r\n\r\nFlexibilit...,"Rare to be recognized, let alone thanked, for ...",Completely casual dress code\r\n\r\nFlexibilit...,-0.15,1.0,"A lot of people complain about the pay, but th...",lot people complain pay pay last company title...,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.256746
0,3.0,1,5.0,"A unique company, but it’s hard to last more t...",2017-04-27,0,-1,1,Completely casual dress code\r\n\r\nFlexibilit...,"Rare to be recognized, let alone thanked, for ...",Completely casual dress code\r\n\r\nFlexibilit...,-0.15,1.0,"If not, there is an ongoing contest where you ...",ongoing contest select take home couple night ...,"[[], [], [], [], [], [], [], [], [], [], [], [...",[“ ”],0.100000


In [35]:
copy_df = df.copy()

In [36]:
copy_df = copy_df.explode('bigrams', ignore_index=False)

In [37]:
res = copy_df[['bigrams', 'polarity']].groupby('bigrams', as_index = False)['polarity'].mean()

In [38]:
print(res)

              bigrams  polarity
0                 * *  0.111442
1            area bay  0.119388
2        balance life  0.180149
3        balance work  0.181769
4   balance work-life  0.135145
5   balance work/life  0.110489
6       benefit great  0.487541
7         cereal free  0.294185
8        change world  0.138785
9            cut edge -0.177305
10           day week  0.129998
11          elon musk  0.217170
12        end quarter  0.061919
13          fast pace  0.247320
14          hour long  0.126206
15          life work  0.178805
16   management upper  0.077670
17          none none  0.154814
18       option stock  0.220048
19                “ ”  0.217626


In [39]:
all_bigrams = res['bigrams'].tolist()
print(all_bigrams)

['* *', 'area bay', 'balance life', 'balance work', 'balance work-life', 'balance work/life', 'benefit great', 'cereal free', 'change world', 'cut edge', 'day week', 'elon musk', 'end quarter', 'fast pace', 'hour long', 'life work', 'management upper', 'none none', 'option stock', '“ ”']


In [40]:
result = copy_df[['bigrams', 'polarity']].groupby('bigrams')['polarity'].mean()

In [41]:
topic_polarities = dict()
for b in all_bigrams:
	topic_polarities[b] = []

In [42]:
polarity_results = []
for i in range(len(quarters)):
	if i < len(quarters) - 1:
		qtr_start = quarters[i]
		qtr_end = quarters[i+1]
		cur_df = df[(df['Date_Posted'] >= qtr_start) & (df['Date_Posted'] < qtr_end)]
		if(len(cur_df) == 0):
			continue
		cur_df = cur_df.explode('bigrams', ignore_index=False)
		results = cur_df[['bigrams', 'polarity']].groupby('bigrams', as_index = False)['polarity'].mean()

		for bigram in all_bigrams:
			if bigram in results['bigrams'].tolist():
				polarity = results[results['bigrams'] == bigram]['polarity'].tolist()[0]
				topic_polarities[bigram].append(polarity)
				# polarity_results.append(results[results['bigrams'] == bigram]['polarity'].tolist()[0])
			else:
				topic_polarities[bigram].append(None)

		# for i, row in results.iterrows():
		# 	if row['bigrams'] in topic_polarities:
		# 		topic_polarities[row['bigrams']].append(row['polarity'])
		# 	else:
		# 		topic_polarities[row['bigrams']] = [row['polarity']]
		polarity_results.append(results)


In [43]:
len(topic_polarities)

20

In [44]:
len(polarity_results)

40

In [45]:
for key, value in topic_polarities.items():
	print(key, len(value))

* * 40
area bay 40
balance life 40
balance work 40
balance work-life 40
balance work/life 40
benefit great 40
cereal free 40
change world 40
cut edge 40
day week 40
elon musk 40
end quarter 40
fast pace 40
hour long 40
life work 40
management upper 40
none none 40
option stock 40
“ ” 40


In [46]:
print(counts)

{'Q1': 6, 'Q2': 6, 'Q3': 3, 'Q4': 6, 'Q5': 6, 'Q6': 12, 'Q7': 6, 'Q8': 7, 'Q9': 16, 'Q10': 13, 'Q11': 29, 'Q12': 23, 'Q13': 35, 'Q14': 47, 'Q15': 75, 'Q16': 72, 'Q17': 66, 'Q18': 57, 'Q19': 85, 'Q20': 81, 'Q21': 68, 'Q22': 79, 'Q23': 118, 'Q24': 138, 'Q25': 153, 'Q26': 157, 'Q27': 203, 'Q28': 142, 'Q29': 221, 'Q30': 189, 'Q31': 170, 'Q32': 135, 'Q33': 121, 'Q34': 99, 'Q35': 183, 'Q36': 253, 'Q37': 506, 'Q38': 379, 'Q39': 387, 'Q40': 321, 'Q41': 0}


In [47]:
final = pd.DataFrame(
{   'quarters': quarts,
	'ratings': normalize_list(avg_star_ratings),
	'title_senti': normalize_list(avg_title_senti),
	'pnc_senti': normalize_list(avg_pnc_senti),
	'work_duration': normalize_list(avg_work_duration),
	'current_employee': avg_current_employee,
	'recommend': avg_recommend,
	'ceo_approval': avg_ceo_approval,
	'business_outlook': avg_business_outlook,
	'recommend_minus': normalize_list(recommend_minus_percentages),
	'ceo_minus': normalize_list(ceo_minus_percentages),
	'business_minus': normalize_list(business_minus_percentages),
	'recommend_unknown': normalize_list(recommend_unknown_percentages),
	'ceo_unknown': normalize_list(ceo_unknown_percentages),
	'business_unknown': normalize_list(business_unknown_percentages),
	'recommend_same': normalize_list(recommend_same_percentages),
	'ceo_same': normalize_list(ceo_same_percentages),
	'business_same': normalize_list(business_same_percentages),
	'recommend_plus': normalize_list(recommend_plus_percentages),
	'ceo_plus': normalize_list(ceo_plus_percentages),
	'business_plus': normalize_list(business_plus_percentages),
})

In [48]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus
0,Q1,0.411765,0.901818,0.484456,0.757057,1,2,1,0,0.363636,0.000000,0.000000,0.468519,0.000000,1.000000,None,1.000000,0.000000,0.333333,0.017778,0.000000
1,Q2,0.102941,0.840000,0.554404,0.757057,1,2,2,0,0.363636,0.500000,0.000000,0.468519,0.000000,0.833333,None,0.250000,0.500000,0.333333,0.508889,0.000000
2,Q3,0.000000,1.000000,0.000000,0.000000,0,-1,2,-1,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,None,0.000000,1.000000,0.000000,0.508889,0.333333
3,Q4,0.926471,0.847273,1.000000,0.738763,1,2,2,2,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,None,0.250000,0.000000,1.000000,0.754444,1.000000
4,Q5,0.102941,0.000000,0.829016,0.928027,1,2,2,2,0.363636,1.000000,0.500000,0.000000,0.000000,0.166667,None,0.000000,1.000000,0.666667,0.508889,0.333333
5,Q6,0.566176,0.883636,0.466321,0.617668,1,2,2,2,0.204545,0.250000,0.750000,0.468519,0.394081,0.166667,None,0.000000,0.250000,0.500000,0.631667,0.500000
6,Q7,0.514706,0.287273,0.391192,0.496574,1,2,2,2,0.363636,0.000000,0.500000,0.000000,0.000000,0.000000,None,0.000000,0.000000,0.666667,1.000000,0.833333
7,Q8,1.000000,0.719481,0.546262,0.214560,1,2,2,2,0.000000,0.428571,0.428571,0.401587,0.337784,0.285714,None,0.000000,0.000000,0.761905,0.579048,0.571429
8,Q9,0.591912,0.333182,0.438148,0.603675,1,2,2,2,0.204545,0.187500,0.375000,0.000000,0.000000,0.062500,None,0.000000,0.375000,0.833333,0.907917,0.687500
9,Q10,0.728507,0.801958,0.707852,0.170015,1,2,2,2,0.167832,0.692308,0.461538,0.216239,0.181884,0.076923,None,0.230769,0.461538,0.717949,0.320000,0.615385


In [49]:
all_bigrams

['* *',
 'area bay',
 'balance life',
 'balance work',
 'balance work-life',
 'balance work/life',
 'benefit great',
 'cereal free',
 'change world',
 'cut edge',
 'day week',
 'elon musk',
 'end quarter',
 'fast pace',
 'hour long',
 'life work',
 'management upper',
 'none none',
 'option stock',
 '“ ”']

In [50]:
for key,value in topic_polarities.items():
	col_name = key.replace(' ', '_')
	final[col_name] = value

In [51]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,*_*,area_bay,balance_life,balance_work,balance_work-life,balance_work/life,benefit_great,cereal_free,change_world,cut_edge,day_week,elon_musk,end_quarter,fast_pace,hour_long,life_work,management_upper,none_none,option_stock,“_”
0,Q1,0.411765,0.901818,0.484456,0.757057,1,2,1,0,0.363636,0.000000,0.000000,0.468519,0.000000,1.000000,None,1.000000,0.000000,0.333333,0.017778,0.000000,-0.014931,NaN,0.500000,0.355208,0.466667,0.243750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.045139,NaN,NaN,NaN,0.045139,NaN
1,Q2,0.102941,0.840000,0.554404,0.757057,1,2,2,0,0.363636,0.500000,0.000000,0.468519,0.000000,0.833333,None,0.250000,0.500000,0.333333,0.508889,0.000000,NaN,NaN,NaN,0.066387,NaN,0.066387,0.800000,0.255415,NaN,NaN,NaN,NaN,NaN,NaN,0.066387,NaN,0.133193,NaN,NaN,NaN
2,Q3,0.000000,1.000000,0.000000,0.000000,0,-1,2,-1,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,None,0.000000,1.000000,0.000000,0.508889,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Q4,0.926471,0.847273,1.000000,0.738763,1,2,2,2,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,None,0.250000,0.000000,1.000000,0.754444,1.000000,NaN,NaN,NaN,0.050000,NaN,0.050000,0.300000,NaN,NaN,-0.081439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Q5,0.102941,0.000000,0.829016,0.928027,1,2,2,2,0.363636,1.000000,0.500000,0.000000,0.000000,0.166667,None,0.000000,1.000000,0.666667,0.508889,0.333333,NaN,NaN,NaN,NaN,NaN,-0.875000,NaN,0.209150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Q6,0.566176,0.883636,0.466321,0.617668,1,2,2,2,0.204545,0.250000,0.750000,0.468519,0.394081,0.166667,None,0.000000,0.250000,0.500000,0.631667,0.500000,NaN,NaN,0.001667,0.001667,NaN,NaN,NaN,NaN,0.010000,0.300000,NaN,0.300000,NaN,0.450000,NaN,0.001667,0.003333,NaN,NaN,NaN
6,Q7,0.514706,0.287273,0.391192,0.496574,1,2,2,2,0.363636,0.000000,0.500000,0.000000,0.000000,0.000000,None,0.000000,0.000000,0.666667,1.000000,0.833333,NaN,NaN,0.083333,0.083333,0.102083,NaN,NaN,NaN,NaN,0.380000,NaN,NaN,NaN,NaN,NaN,0.083333,NaN,NaN,NaN,0.100595
7,Q8,1.000000,0.719481,0.546262,0.214560,1,2,2,2,0.000000,0.428571,0.428571,0.401587,0.337784,0.285714,None,0.000000,0.000000,0.761905,0.579048,0.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.081250,NaN,NaN,NaN,NaN,0.337778,NaN,NaN,NaN,NaN,NaN
8,Q9,0.591912,0.333182,0.438148,0.603675,1,2,2,2,0.204545,0.187500,0.375000,0.000000,0.000000,0.062500,None,0.000000,0.375000,0.833333,0.907917,0.687500,NaN,NaN,0.128571,0.128571,NaN,NaN,NaN,NaN,NaN,-0.101190,NaN,NaN,NaN,NaN,0.116667,0.128571,NaN,0.600000,0.000000,NaN
9,Q10,0.728507,0.801958,0.707852,0.170015,1,2,2,2,0.167832,0.692308,0.461538,0.216239,0.181884,0.076923,None,0.230769,0.461538,0.717949,0.320000,0.615385,0.166071,NaN,NaN,0.079167,NaN,-0.875000,0.550000,0.159077,NaN,NaN,NaN,NaN,-0.025000,NaN,0.106746,NaN,0.200000,NaN,NaN,NaN


In [52]:
for key, value in topic_polarities.items():
	col = key.replace(' ', '_')
	final[col].fillna(final[col].mean(),inplace=True)

In [53]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,*_*,area_bay,balance_life,balance_work,balance_work-life,balance_work/life,benefit_great,cereal_free,change_world,cut_edge,day_week,elon_musk,end_quarter,fast_pace,hour_long,life_work,management_upper,none_none,option_stock,“_”
0,Q1,0.411765,0.901818,0.484456,0.757057,1,2,1,0,0.363636,0.000000,0.000000,0.468519,0.000000,1.000000,None,1.000000,0.000000,0.333333,0.017778,0.000000,-0.014931,0.106592,0.500000,0.355208,0.466667,0.243750,0.506491,0.284236,0.122694,-0.164135,0.112804,0.194127,0.104738,0.273807,0.045139,0.155164,0.082187,0.171604,0.045139,0.281928
1,Q2,0.102941,0.840000,0.554404,0.757057,1,2,2,0,0.363636,0.500000,0.000000,0.468519,0.000000,0.833333,None,0.250000,0.500000,0.333333,0.508889,0.000000,0.138875,0.106592,0.177913,0.066387,0.184352,0.066387,0.800000,0.255415,0.122694,-0.164135,0.112804,0.194127,0.104738,0.273807,0.066387,0.155164,0.133193,0.171604,0.218785,0.281928
2,Q3,0.000000,1.000000,0.000000,0.000000,0,-1,2,-1,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,None,0.000000,1.000000,0.000000,0.508889,0.333333,0.138875,0.106592,0.177913,0.146881,0.184352,0.075581,0.506491,0.284236,0.122694,-0.600000,0.112804,0.194127,0.104738,0.273807,0.108998,0.155164,0.082187,0.171604,0.218785,0.281928
3,Q4,0.926471,0.847273,1.000000,0.738763,1,2,2,2,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,None,0.250000,0.000000,1.000000,0.754444,1.000000,0.138875,0.106592,0.177913,0.050000,0.184352,0.050000,0.300000,0.284236,0.122694,-0.081439,0.112804,0.194127,0.104738,0.273807,0.108998,0.155164,0.082187,0.171604,0.218785,0.281928
4,Q5,0.102941,0.000000,0.829016,0.928027,1,2,2,2,0.363636,1.000000,0.500000,0.000000,0.000000,0.166667,None,0.000000,1.000000,0.666667,0.508889,0.333333,0.138875,0.106592,0.177913,0.146881,0.184352,-0.875000,0.506491,0.209150,0.122694,-0.164135,0.112804,0.194127,0.104738,0.273807,0.108998,0.155164,0.082187,0.171604,0.218785,0.281928
5,Q6,0.566176,0.883636,0.466321,0.617668,1,2,2,2,0.204545,0.250000,0.750000,0.468519,0.394081,0.166667,None,0.000000,0.250000,0.500000,0.631667,0.500000,0.138875,0.106592,0.001667,0.001667,0.184352,0.075581,0.506491,0.284236,0.010000,0.300000,0.112804,0.300000,0.104738,0.450000,0.108998,0.001667,0.003333,0.171604,0.218785,0.281928
6,Q7,0.514706,0.287273,0.391192,0.496574,1,2,2,2,0.363636,0.000000,0.500000,0.000000,0.000000,0.000000,None,0.000000,0.000000,0.666667,1.000000,0.833333,0.138875,0.106592,0.083333,0.083333,0.102083,0.075581,0.506491,0.284236,0.122694,0.380000,0.112804,0.194127,0.104738,0.273807,0.108998,0.083333,0.082187,0.171604,0.218785,0.100595
7,Q8,1.000000,0.719481,0.546262,0.214560,1,2,2,2,0.000000,0.428571,0.428571,0.401587,0.337784,0.285714,None,0.000000,0.000000,0.761905,0.579048,0.571429,0.138875,0.106592,0.177913,0.146881,0.184352,0.075581,0.506491,0.284236,0.122694,-0.081250,0.112804,0.194127,0.104738,0.273807,0.337778,0.155164,0.082187,0.171604,0.218785,0.281928
8,Q9,0.591912,0.333182,0.438148,0.603675,1,2,2,2,0.204545,0.187500,0.375000,0.000000,0.000000,0.062500,None,0.000000,0.375000,0.833333,0.907917,0.687500,0.138875,0.106592,0.128571,0.128571,0.184352,0.075581,0.506491,0.284236,0.122694,-0.101190,0.112804,0.194127,0.104738,0.273807,0.116667,0.128571,0.082187,0.600000,0.000000,0.281928
9,Q10,0.728507,0.801958,0.707852,0.170015,1,2,2,2,0.167832,0.692308,0.461538,0.216239,0.181884,0.076923,None,0.230769,0.461538,0.717949,0.320000,0.615385,0.166071,0.106592,0.177913,0.079167,0.184352,-0.875000,0.550000,0.159077,0.122694,-0.164135,0.112804,0.194127,-0.025000,0.273807,0.106746,0.155164,0.200000,0.171604,0.218785,0.281928


In [54]:
len(final)

40

In [55]:
earnings = [
2321,
1618,
1142,
438,
301,
300,
104,
16,
105,
143,
-408,
-702,
140,
311,
-717.54,
-709.55,
-675.95,
-619.38,
-336.4,
-330.28,
-121.34,
21.88,
-293.19,
-282.27,
-320.4,
-229.86,
-184.23,
-154.18,
-107.63,
-74.71,
-61.9,
-49.8,
-16.26,
-38.5,
-30.5,
11.25,
-89.93,
-110.8,
-105.6,
-89.87
]

earnings = earnings[::-1]

In [56]:
len(earnings)

40

In [57]:
final['earnings'] = earnings

In [58]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,*_*,area_bay,balance_life,balance_work,balance_work-life,balance_work/life,benefit_great,cereal_free,change_world,cut_edge,day_week,elon_musk,end_quarter,fast_pace,hour_long,life_work,management_upper,none_none,option_stock,“_”,earnings
0,Q1,0.411765,0.901818,0.484456,0.757057,1,2,1,0,0.363636,0.000000,0.000000,0.468519,0.000000,1.000000,None,1.000000,0.000000,0.333333,0.017778,0.000000,-0.014931,0.106592,0.500000,0.355208,0.466667,0.243750,0.506491,0.284236,0.122694,-0.164135,0.112804,0.194127,0.104738,0.273807,0.045139,0.155164,0.082187,0.171604,0.045139,0.281928,-89.87
1,Q2,0.102941,0.840000,0.554404,0.757057,1,2,2,0,0.363636,0.500000,0.000000,0.468519,0.000000,0.833333,None,0.250000,0.500000,0.333333,0.508889,0.000000,0.138875,0.106592,0.177913,0.066387,0.184352,0.066387,0.800000,0.255415,0.122694,-0.164135,0.112804,0.194127,0.104738,0.273807,0.066387,0.155164,0.133193,0.171604,0.218785,0.281928,-105.60
2,Q3,0.000000,1.000000,0.000000,0.000000,0,-1,2,-1,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,None,0.000000,1.000000,0.000000,0.508889,0.333333,0.138875,0.106592,0.177913,0.146881,0.184352,0.075581,0.506491,0.284236,0.122694,-0.600000,0.112804,0.194127,0.104738,0.273807,0.108998,0.155164,0.082187,0.171604,0.218785,0.281928,-110.80
3,Q4,0.926471,0.847273,1.000000,0.738763,1,2,2,2,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,None,0.250000,0.000000,1.000000,0.754444,1.000000,0.138875,0.106592,0.177913,0.050000,0.184352,0.050000,0.300000,0.284236,0.122694,-0.081439,0.112804,0.194127,0.104738,0.273807,0.108998,0.155164,0.082187,0.171604,0.218785,0.281928,-89.93
4,Q5,0.102941,0.000000,0.829016,0.928027,1,2,2,2,0.363636,1.000000,0.500000,0.000000,0.000000,0.166667,None,0.000000,1.000000,0.666667,0.508889,0.333333,0.138875,0.106592,0.177913,0.146881,0.184352,-0.875000,0.506491,0.209150,0.122694,-0.164135,0.112804,0.194127,0.104738,0.273807,0.108998,0.155164,0.082187,0.171604,0.218785,0.281928,11.25
5,Q6,0.566176,0.883636,0.466321,0.617668,1,2,2,2,0.204545,0.250000,0.750000,0.468519,0.394081,0.166667,None,0.000000,0.250000,0.500000,0.631667,0.500000,0.138875,0.106592,0.001667,0.001667,0.184352,0.075581,0.506491,0.284236,0.010000,0.300000,0.112804,0.300000,0.104738,0.450000,0.108998,0.001667,0.003333,0.171604,0.218785,0.281928,-30.50
6,Q7,0.514706,0.287273,0.391192,0.496574,1,2,2,2,0.363636,0.000000,0.500000,0.000000,0.000000,0.000000,None,0.000000,0.000000,0.666667,1.000000,0.833333,0.138875,0.106592,0.083333,0.083333,0.102083,0.075581,0.506491,0.284236,0.122694,0.380000,0.112804,0.194127,0.104738,0.273807,0.108998,0.083333,0.082187,0.171604,0.218785,0.100595,-38.50
7,Q8,1.000000,0.719481,0.546262,0.214560,1,2,2,2,0.000000,0.428571,0.428571,0.401587,0.337784,0.285714,None,0.000000,0.000000,0.761905,0.579048,0.571429,0.138875,0.106592,0.177913,0.146881,0.184352,0.075581,0.506491,0.284236,0.122694,-0.081250,0.112804,0.194127,0.104738,0.273807,0.337778,0.155164,0.082187,0.171604,0.218785,0.281928,-16.26
8,Q9,0.591912,0.333182,0.438148,0.603675,1,2,2,2,0.204545,0.187500,0.375000,0.000000,0.000000,0.062500,None,0.000000,0.375000,0.833333,0.907917,0.687500,0.138875,0.106592,0.128571,0.128571,0.184352,0.075581,0.506491,0.284236,0.122694,-0.101190,0.112804,0.194127,0.104738,0.273807,0.116667,0.128571,0.082187,0.600000,0.000000,0.281928,-49.80
9,Q10,0.728507,0.801958,0.707852,0.170015,1,2,2,2,0.167832,0.692308,0.461538,0.216239,0.181884,0.076923,None,0.230769,0.461538,0.717949,0.320000,0.615385,0.166071,0.106592,0.177913,0.079167,0.184352,-0.875000,0.550000,0.159077,0.122694,-0.164135,0.112804,0.194127,-0.025000,0.273807,0.106746,0.155164,0.200000,0.171604,0.218785,0.281928,-61.90


In [59]:
pred_column = [None]
for i in range(1, len(earnings)):
	if earnings[i] > earnings[i-1]:
		pred_column.append(1)
	else:
		pred_column.append(0)

In [60]:
pred_column

[None,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [61]:
final['pred_column'] = pred_column

In [62]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,*_*,area_bay,balance_life,balance_work,balance_work-life,balance_work/life,benefit_great,cereal_free,change_world,cut_edge,day_week,elon_musk,end_quarter,fast_pace,hour_long,life_work,management_upper,none_none,option_stock,“_”,earnings,pred_column
0,Q1,0.411765,0.901818,0.484456,0.757057,1,2,1,0,0.363636,0.000000,0.000000,0.468519,0.000000,1.000000,None,1.000000,0.000000,0.333333,0.017778,0.000000,-0.014931,0.106592,0.500000,0.355208,0.466667,0.243750,0.506491,0.284236,0.122694,-0.164135,0.112804,0.194127,0.104738,0.273807,0.045139,0.155164,0.082187,0.171604,0.045139,0.281928,-89.87,NaN
1,Q2,0.102941,0.840000,0.554404,0.757057,1,2,2,0,0.363636,0.500000,0.000000,0.468519,0.000000,0.833333,None,0.250000,0.500000,0.333333,0.508889,0.000000,0.138875,0.106592,0.177913,0.066387,0.184352,0.066387,0.800000,0.255415,0.122694,-0.164135,0.112804,0.194127,0.104738,0.273807,0.066387,0.155164,0.133193,0.171604,0.218785,0.281928,-105.60,0.0
2,Q3,0.000000,1.000000,0.000000,0.000000,0,-1,2,-1,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,None,0.000000,1.000000,0.000000,0.508889,0.333333,0.138875,0.106592,0.177913,0.146881,0.184352,0.075581,0.506491,0.284236,0.122694,-0.600000,0.112804,0.194127,0.104738,0.273807,0.108998,0.155164,0.082187,0.171604,0.218785,0.281928,-110.80,0.0
3,Q4,0.926471,0.847273,1.000000,0.738763,1,2,2,2,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,None,0.250000,0.000000,1.000000,0.754444,1.000000,0.138875,0.106592,0.177913,0.050000,0.184352,0.050000,0.300000,0.284236,0.122694,-0.081439,0.112804,0.194127,0.104738,0.273807,0.108998,0.155164,0.082187,0.171604,0.218785,0.281928,-89.93,1.0
4,Q5,0.102941,0.000000,0.829016,0.928027,1,2,2,2,0.363636,1.000000,0.500000,0.000000,0.000000,0.166667,None,0.000000,1.000000,0.666667,0.508889,0.333333,0.138875,0.106592,0.177913,0.146881,0.184352,-0.875000,0.506491,0.209150,0.122694,-0.164135,0.112804,0.194127,0.104738,0.273807,0.108998,0.155164,0.082187,0.171604,0.218785,0.281928,11.25,1.0
5,Q6,0.566176,0.883636,0.466321,0.617668,1,2,2,2,0.204545,0.250000,0.750000,0.468519,0.394081,0.166667,None,0.000000,0.250000,0.500000,0.631667,0.500000,0.138875,0.106592,0.001667,0.001667,0.184352,0.075581,0.506491,0.284236,0.010000,0.300000,0.112804,0.300000,0.104738,0.450000,0.108998,0.001667,0.003333,0.171604,0.218785,0.281928,-30.50,0.0
6,Q7,0.514706,0.287273,0.391192,0.496574,1,2,2,2,0.363636,0.000000,0.500000,0.000000,0.000000,0.000000,None,0.000000,0.000000,0.666667,1.000000,0.833333,0.138875,0.106592,0.083333,0.083333,0.102083,0.075581,0.506491,0.284236,0.122694,0.380000,0.112804,0.194127,0.104738,0.273807,0.108998,0.083333,0.082187,0.171604,0.218785,0.100595,-38.50,0.0
7,Q8,1.000000,0.719481,0.546262,0.214560,1,2,2,2,0.000000,0.428571,0.428571,0.401587,0.337784,0.285714,None,0.000000,0.000000,0.761905,0.579048,0.571429,0.138875,0.106592,0.177913,0.146881,0.184352,0.075581,0.506491,0.284236,0.122694,-0.081250,0.112804,0.194127,0.104738,0.273807,0.337778,0.155164,0.082187,0.171604,0.218785,0.281928,-16.26,1.0
8,Q9,0.591912,0.333182,0.438148,0.603675,1,2,2,2,0.204545,0.187500,0.375000,0.000000,0.000000,0.062500,None,0.000000,0.375000,0.833333,0.907917,0.687500,0.138875,0.106592,0.128571,0.128571,0.184352,0.075581,0.506491,0.284236,0.122694,-0.101190,0.112804,0.194127,0.104738,0.273807,0.116667,0.128571,0.082187,0.600000,0.000000,0.281928,-49.80,0.0
9,Q10,0.728507,0.801958,0.707852,0.170015,1,2,2,2,0.167832,0.692308,0.461538,0.216239,0.181884,0.076923,None,0.230769,0.461538,0.717949,0.320000,0.615385,0.166071,0.106592,0.177913,0.079167,0.184352,-0.875000,0.550000,0.159077,0.122694,-0.164135,0.112804,0.194127,-0.025000,0.273807,0.106746,0.155164,0.200000,0.171604,0.218785,0.281928,-61.90,0.0


In [63]:
final.drop(['recommend_same'], axis=1, inplace=True)

In [64]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,*_*,area_bay,balance_life,balance_work,balance_work-life,balance_work/life,benefit_great,cereal_free,change_world,cut_edge,day_week,elon_musk,end_quarter,fast_pace,hour_long,life_work,management_upper,none_none,option_stock,“_”,earnings,pred_column
0,Q1,0.411765,0.901818,0.484456,0.757057,1,2,1,0,0.363636,0.000000,0.000000,0.468519,0.000000,1.000000,1.000000,0.000000,0.333333,0.017778,0.000000,-0.014931,0.106592,0.500000,0.355208,0.466667,0.243750,0.506491,0.284236,0.122694,-0.164135,0.112804,0.194127,0.104738,0.273807,0.045139,0.155164,0.082187,0.171604,0.045139,0.281928,-89.87,NaN
1,Q2,0.102941,0.840000,0.554404,0.757057,1,2,2,0,0.363636,0.500000,0.000000,0.468519,0.000000,0.833333,0.250000,0.500000,0.333333,0.508889,0.000000,0.138875,0.106592,0.177913,0.066387,0.184352,0.066387,0.800000,0.255415,0.122694,-0.164135,0.112804,0.194127,0.104738,0.273807,0.066387,0.155164,0.133193,0.171604,0.218785,0.281928,-105.60,0.0
2,Q3,0.000000,1.000000,0.000000,0.000000,0,-1,2,-1,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.508889,0.333333,0.138875,0.106592,0.177913,0.146881,0.184352,0.075581,0.506491,0.284236,0.122694,-0.600000,0.112804,0.194127,0.104738,0.273807,0.108998,0.155164,0.082187,0.171604,0.218785,0.281928,-110.80,0.0
3,Q4,0.926471,0.847273,1.000000,0.738763,1,2,2,2,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,1.000000,0.754444,1.000000,0.138875,0.106592,0.177913,0.050000,0.184352,0.050000,0.300000,0.284236,0.122694,-0.081439,0.112804,0.194127,0.104738,0.273807,0.108998,0.155164,0.082187,0.171604,0.218785,0.281928,-89.93,1.0
4,Q5,0.102941,0.000000,0.829016,0.928027,1,2,2,2,0.363636,1.000000,0.500000,0.000000,0.000000,0.166667,0.000000,1.000000,0.666667,0.508889,0.333333,0.138875,0.106592,0.177913,0.146881,0.184352,-0.875000,0.506491,0.209150,0.122694,-0.164135,0.112804,0.194127,0.104738,0.273807,0.108998,0.155164,0.082187,0.171604,0.218785,0.281928,11.25,1.0
5,Q6,0.566176,0.883636,0.466321,0.617668,1,2,2,2,0.204545,0.250000,0.750000,0.468519,0.394081,0.166667,0.000000,0.250000,0.500000,0.631667,0.500000,0.138875,0.106592,0.001667,0.001667,0.184352,0.075581,0.506491,0.284236,0.010000,0.300000,0.112804,0.300000,0.104738,0.450000,0.108998,0.001667,0.003333,0.171604,0.218785,0.281928,-30.50,0.0
6,Q7,0.514706,0.287273,0.391192,0.496574,1,2,2,2,0.363636,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,1.000000,0.833333,0.138875,0.106592,0.083333,0.083333,0.102083,0.075581,0.506491,0.284236,0.122694,0.380000,0.112804,0.194127,0.104738,0.273807,0.108998,0.083333,0.082187,0.171604,0.218785,0.100595,-38.50,0.0
7,Q8,1.000000,0.719481,0.546262,0.214560,1,2,2,2,0.000000,0.428571,0.428571,0.401587,0.337784,0.285714,0.000000,0.000000,0.761905,0.579048,0.571429,0.138875,0.106592,0.177913,0.146881,0.184352,0.075581,0.506491,0.284236,0.122694,-0.081250,0.112804,0.194127,0.104738,0.273807,0.337778,0.155164,0.082187,0.171604,0.218785,0.281928,-16.26,1.0
8,Q9,0.591912,0.333182,0.438148,0.603675,1,2,2,2,0.204545,0.187500,0.375000,0.000000,0.000000,0.062500,0.000000,0.375000,0.833333,0.907917,0.687500,0.138875,0.106592,0.128571,0.128571,0.184352,0.075581,0.506491,0.284236,0.122694,-0.101190,0.112804,0.194127,0.104738,0.273807,0.116667,0.128571,0.082187,0.600000,0.000000,0.281928,-49.80,0.0
9,Q10,0.728507,0.801958,0.707852,0.170015,1,2,2,2,0.167832,0.692308,0.461538,0.216239,0.181884,0.076923,0.230769,0.461538,0.717949,0.320000,0.615385,0.166071,0.106592,0.177913,0.079167,0.184352,-0.875000,0.550000,0.159077,0.122694,-0.164135,0.112804,0.194127,-0.025000,0.273807,0.106746,0.155164,0.200000,0.171604,0.218785,0.281928,-61.90,0.0


In [65]:
final.dropna(inplace=True)

In [66]:
final.drop(['recommend_unknown', 'ceo_unknown', 'business_unknown'], axis=1, inplace=True)

In [67]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,*_*,area_bay,balance_life,balance_work,balance_work-life,balance_work/life,benefit_great,cereal_free,change_world,cut_edge,day_week,elon_musk,end_quarter,fast_pace,hour_long,life_work,management_upper,none_none,option_stock,“_”,earnings,pred_column
1,Q2,0.102941,0.840000,0.554404,0.757057,1,2,2,0,0.363636,0.500000,0.000000,0.250000,0.500000,0.333333,0.508889,0.000000,0.138875,0.106592,0.177913,0.066387,0.184352,0.066387,0.800000,0.255415,0.122694,-0.164135,0.112804,0.194127,0.104738,0.273807,0.066387,0.155164,0.133193,0.171604,0.218785,0.281928,-105.60,0.0
2,Q3,0.000000,1.000000,0.000000,0.000000,0,-1,2,-1,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.508889,0.333333,0.138875,0.106592,0.177913,0.146881,0.184352,0.075581,0.506491,0.284236,0.122694,-0.600000,0.112804,0.194127,0.104738,0.273807,0.108998,0.155164,0.082187,0.171604,0.218785,0.281928,-110.80,0.0
3,Q4,0.926471,0.847273,1.000000,0.738763,1,2,2,2,0.045455,0.000000,0.000000,0.250000,0.000000,1.000000,0.754444,1.000000,0.138875,0.106592,0.177913,0.050000,0.184352,0.050000,0.300000,0.284236,0.122694,-0.081439,0.112804,0.194127,0.104738,0.273807,0.108998,0.155164,0.082187,0.171604,0.218785,0.281928,-89.93,1.0
4,Q5,0.102941,0.000000,0.829016,0.928027,1,2,2,2,0.363636,1.000000,0.500000,0.000000,1.000000,0.666667,0.508889,0.333333,0.138875,0.106592,0.177913,0.146881,0.184352,-0.875000,0.506491,0.209150,0.122694,-0.164135,0.112804,0.194127,0.104738,0.273807,0.108998,0.155164,0.082187,0.171604,0.218785,0.281928,11.25,1.0
5,Q6,0.566176,0.883636,0.466321,0.617668,1,2,2,2,0.204545,0.250000,0.750000,0.000000,0.250000,0.500000,0.631667,0.500000,0.138875,0.106592,0.001667,0.001667,0.184352,0.075581,0.506491,0.284236,0.010000,0.300000,0.112804,0.300000,0.104738,0.450000,0.108998,0.001667,0.003333,0.171604,0.218785,0.281928,-30.50,0.0
6,Q7,0.514706,0.287273,0.391192,0.496574,1,2,2,2,0.363636,0.000000,0.500000,0.000000,0.000000,0.666667,1.000000,0.833333,0.138875,0.106592,0.083333,0.083333,0.102083,0.075581,0.506491,0.284236,0.122694,0.380000,0.112804,0.194127,0.104738,0.273807,0.108998,0.083333,0.082187,0.171604,0.218785,0.100595,-38.50,0.0
7,Q8,1.000000,0.719481,0.546262,0.214560,1,2,2,2,0.000000,0.428571,0.428571,0.000000,0.000000,0.761905,0.579048,0.571429,0.138875,0.106592,0.177913,0.146881,0.184352,0.075581,0.506491,0.284236,0.122694,-0.081250,0.112804,0.194127,0.104738,0.273807,0.337778,0.155164,0.082187,0.171604,0.218785,0.281928,-16.26,1.0
8,Q9,0.591912,0.333182,0.438148,0.603675,1,2,2,2,0.204545,0.187500,0.375000,0.000000,0.375000,0.833333,0.907917,0.687500,0.138875,0.106592,0.128571,0.128571,0.184352,0.075581,0.506491,0.284236,0.122694,-0.101190,0.112804,0.194127,0.104738,0.273807,0.116667,0.128571,0.082187,0.600000,0.000000,0.281928,-49.80,0.0
9,Q10,0.728507,0.801958,0.707852,0.170015,1,2,2,2,0.167832,0.692308,0.461538,0.230769,0.461538,0.717949,0.320000,0.615385,0.166071,0.106592,0.177913,0.079167,0.184352,-0.875000,0.550000,0.159077,0.122694,-0.164135,0.112804,0.194127,-0.025000,0.273807,0.106746,0.155164,0.200000,0.171604,0.218785,0.281928,-61.90,0.0
10,Q11,0.376268,0.510219,0.777202,0.418964,1,2,2,2,0.188088,0.000000,0.103448,0.310345,0.724138,0.781609,0.644368,0.724138,0.138875,0.625000,0.029167,0.072880,0.202865,0.075581,0.335795,0.400000,-0.037879,-0.600000,0.112804,0.113333,0.104738,0.324236,0.126894,0.038889,0.084028,0.066667,0.317647,0.281928,-74.71,0.0


In [71]:
final.to_excel('tesla_cleaned_data.xlsx')

In [68]:
final.to_csv('tesla_cleaned_data_csv.csv')